In [20]:
# =============================================================================
# data_cleaning.ipynb
# Nettoyage complet du dataset ChestX-ray14 (NIH Clinical Center)
# Objectif : produire un CSV propre avec labels multi-hot (one-hot par pathologie)
#           + suppression des images manquantes + filtrage cohérent
# Auteur : [Ton nom]
# Date   : 2025
# =============================================================================

import os
import pandas as pd
import numpy as np
import logging
from pathlib import Path

In [21]:
# ----------------------------- Configuration ---------------------------------
PROJECT_ROOT = Path.cwd().parent  # Plus fiable que '../'
DATA_RAW = PROJECT_ROOT / 'data' / 'raw'
DATA_PROCESSED = PROJECT_ROOT / 'data' / 'processed'
LOG_DIR = PROJECT_ROOT / 'logs'

DATA_PROCESSED.mkdir(parents=True, exist_ok=True)
LOG_DIR.mkdir(parents=True, exist_ok=True)

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(LOG_DIR / 'data_cleaning.log', encoding='utf-8'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)
logger = logging.getLogger(__name__)



In [22]:
# ----------------------------- Chemins ---------------------------------
raw_csv_path = DATA_RAW / 'Data_Entry_2017.csv'
images_dir = DATA_RAW / 'images'
clean_csv_path = DATA_PROCESSED / 'chestxray14_clean.csv'  # nom plus clair

logger.info("Démarrage du nettoyage ChestX-ray14")

2025-12-08 20:53:17,465 - INFO - Démarrage du nettoyage ChestX-ray14


In [23]:
# ----------------------------- 1. Chargement --------------------------------
if not raw_csv_path.exists():
    raise FileNotFoundError(f"CSV manquant : {raw_csv_path}")
df = pd.read_csv(raw_csv_path)
logger.info(f"CSV chargé : {df.shape}")

# ----------------------------- 2. Colonnes utiles ----------------------------
df = df[['Image Index', 'Finding Labels', 'Patient Age', 'Patient Gender', 'Patient ID']].copy()

# ----------------------------- 3. Nettoyage de base -------------------------
df.drop_duplicates(subset='Image Index', inplace=True)
df['Finding Labels'] = df['Finding Labels'].str.replace('No Finding', 'Normal')

# Nettoyage âge + genre
df['Patient Age'] = pd.to_numeric(df['Patient Age'], errors='coerce')
df = df[df['Patient Age'].between(1, 100)]  # 100 ans max, plus réaliste (évite outliers >200)
df = df[df['Patient Gender'].isin(['M', 'F'])]

# ----------------------------- 4. Vérification images présentes -------------
image_files = set(f for f in os.listdir(images_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg')))
df_images = set(df['Image Index'])
missing = df_images - image_files

if missing:
    logger.warning(f"{len(missing)} images manquantes → supprimées")
    with open(LOG_DIR / 'missing_images.txt', 'w') as f:
        f.write('\n'.join(sorted(missing)))
    df = df[df['Image Index'].isin(image_files)]
logger.info(f"Images valides après filtrage : {len(df)}")

# ----------------------------- 5. Multi-label one-hot (CORRECTION IMPORTANTE) ---
# Liste officielle NIH des 14 + Normal
PATHOLOGIES = [
    'Atelectasis', 'Consolidation', 'Infiltration', 'Pneumothorax', 'Edema',
    'Emphysema', 'Fibrosis', 'Effusion', 'Pneumonia', 'Pleural_Thickening',
    'Cardiomegaly', 'Nodule', 'Mass', 'Hernia', 'Normal'
]

# Encodage propre (évite les bugs de split sur chaînes vides)
for path in PATHOLOGIES:
    df[path] = df['Finding Labels'].str.contains(path, case=False).astype(int)

# ----------------------------- 6. Statistiques -----------------------------
logger.info("=== RÉPARTITION FINALE ===")
for path in PATHOLOGIES:
    count = df[path].sum()
    logger.info(f"{path:20} : {count:>6} cas ({count/len(df)*100:5.2f}%)")

2025-12-08 20:53:17,804 - INFO - CSV chargé : (112120, 12)
2025-12-08 20:53:17,982 - WARNING - 107105 images manquantes → supprimées
2025-12-08 20:53:18,099 - INFO - Images valides après filtrage : 4999
2025-12-08 20:53:18,154 - INFO - === RÉPARTITION FINALE ===
2025-12-08 20:53:18,156 - INFO - Atelectasis          :    460 cas ( 9.20%)
2025-12-08 20:53:18,157 - INFO - Consolidation        :    205 cas ( 4.10%)
2025-12-08 20:53:18,160 - INFO - Infiltration         :    830 cas (16.60%)
2025-12-08 20:53:18,163 - INFO - Pneumothorax         :    199 cas ( 3.98%)
2025-12-08 20:53:18,166 - INFO - Edema                :     90 cas ( 1.80%)
2025-12-08 20:53:18,168 - INFO - Emphysema            :    125 cas ( 2.50%)
2025-12-08 20:53:18,170 - INFO - Fibrosis             :    172 cas ( 3.44%)
2025-12-08 20:53:18,173 - INFO - Effusion             :    487 cas ( 9.74%)
2025-12-08 20:53:18,176 - INFO - Pneumonia            :     65 cas ( 1.30%)
2025-12-08 20:53:18,179 - INFO - Pleural_Thickening  

In [24]:
# ----------------------------- 7. Sauvegarde -------------------------------
final_cols = ['Image Index', 'Patient ID', 'Patient Age', 'Patient Gender'] + PATHOLOGIES
df_final = df[final_cols].reset_index(drop=True)
df_final.to_csv(clean_csv_path, index=False)

logger.info(f"Nettoyage terminé ! → {clean_csv_path}")
logger.info(f"Images finales : {len(df_final)}")
print("NETTOYAGE TERMINÉ - FICHIER PRÊT !")
print(f"Sauvegardé : {clean_csv_path}")

2025-12-08 20:53:18,275 - INFO - Nettoyage terminé ! → c:\Users\Asus\Diagnostic-Radiologique\data\processed\chestxray14_clean.csv
2025-12-08 20:53:18,279 - INFO - Images finales : 4999


NETTOYAGE TERMINÉ - FICHIER PRÊT !
Sauvegardé : c:\Users\Asus\Diagnostic-Radiologique\data\processed\chestxray14_clean.csv
